In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
team_ref = pd.read_csv('datasets/teams.csv')

In [3]:
team_ref

,team,city,province,short,colour,crest,link,wiki
0,Atletico Ottawa,Ottawa,Ontario,AO,w3-vivid-red,Atletico_Ottawa_nav.png,https://atleticoottawa.canpl.ca/,https://en.wikipedia.org/wiki/Atl%C3%A9tico_Ot...
1,Cavalry FC,Calgary,Alberta,CFC,w3-2019-fiesta,cavalry_fc_nav.png,https://cavalryfc.canpl.ca/,https://en.wikipedia.org/wiki/Cavalry_FC
2,FC Edmonton,Edmonton,Alberta,FCE,w3-2019-princess-blue,FC_Edmonton_nav.png,https://fcedmonton.canpl.ca/,https://en.wikipedia.org/wiki/FC_Edmonton
3,Forge FC,Hamilton,Ontario,FFC,w3-2019-turmeric,Forge_FC_nav.png,https://forgefc.canpl.ca/,https://en.wikipedia.org/wiki/Forge_FC
4,HFX Wanderers FC,Halifax,Nova Scotia,HFX,w3-vivid-blue,HFX_Wanderers_FC.png,https://hfxwanderersfc.canpl.ca/,https://en.wikipedia.org/wiki/HFX_Wanderers_FC
5,Pacific FC,Langford,British Columbia,PFC,w3-vivid-reddish-purple,Pacific_FC_nav.png,https://pacificfc.canpl.ca/,https://en.wikipedia.org/wiki/Pacific_FC
6,Valour FC,Winnipeg,Manitoba,VFC,w3-2019-biking-red,Valour_FC_nav.png,https://valourfc.canpl.ca/,https://en.wikipedia.org/wiki/Valour_FC
7,York9 FC,York Region,Ontario,Y9,w3-vivid-yellow-green,York_9_FC_nav.png,https://york9fc.canpl.ca/,https://en.wikipedia.org/wiki/York9_FC


In [4]:
year = '2019'

In [41]:
results = pd.read_csv(f'datasets/{year}/cpl-{year}-results.csv')
stats = pd.read_csv(f'datasets/{year}/cpl-{year}-stats.csv')
player_info = pd.read_csv(f'datasets/{year}/player-{year}-info.csv')

In [6]:
if year == '2019':
    team_ref = team_ref[1:]
    results_old = results[:-7].copy()
else:
    results_old = results[results['hr'] != 'E'].copy()

In [7]:
def get_long_name(string,team_ref):
    for short in team_ref['short']:
        row = team_ref[team_ref['short'] == short]
        if string == short:
            string = row.iloc[0]['team']
    return string

def get_shortest_name(string,team_ref):
    for team in team_ref['team']:
        row = team_ref[team_ref['team'] == team]
        if string == team:
            string = str(row.iloc[0]['short'])
    return string

def get_schedule(data):
    db = data.copy()
    db = db[db['s'] <= 1]
    #db = db.tail(4)
    db = db[['game','home','away']]
    db = index_reset(db)
    db = db.sort_values(by=['game'])
    return db

In [8]:
def fix_db_na(data):
    db = data.copy()
    if db['team'].isnull().values.any():
        for row in range(db.shape[0]):
            if pd.isna(db.iloc[row]['team']) == True:
                print(True)
                db.iloc[row]['team'] = get_long_name(db.iloc[row]['team'],data)
    return db

In [9]:
results.tail(5)

,game,s,d,m,hs,as,home,hr,away,ar,csh,csa,combined,venue,links
95,I96,1,19,10,0,2,York9 FC,L,HFX Wanderers FC,W,0,1,10-19-2019 York9 FC L 0-2 W HFX Wanderers FC,York Lions Stadium,https://canpl.ca/matchcentre/5opkqc01qgmjdcbho...
96,I97,1,19,10,3,1,Cavalry FC,W,FC Edmonton,L,0,0,10-19-2019 Cavalry FC W 3-1 L FC Edmonton,ATCO Field,https://canpl.ca/matchcentre/5nw6ub2q95ptd6n71...
97,I98,1,19,10,2,0,Pacific FC,W,Valour FC,L,1,0,10-19-2019 Pacific FC W 2-0 L Valour FC,Westhills Stadium,https://canpl.ca/matchcentre/5opkqc01qgmjdcbho...
98,I99,2,26,10,1,0,Forge FC,W,Cavalry FC,L,1,0,10-26-2019 Forge FC W 1-0 L Cavalry FC,Tim Hortons Field,https://canpl.ca/matchcentre/7exzoqwijjljpmysg...
99,I100,2,2,11,0,1,Cavalry FC,L,Forge FC,W,0,1,11-2-2019 Cavalry FC L 0-1 W Forge FC,ATCO Field,https://canpl.ca/matchcentre/7f2xl06opfb8koy1j...


In [10]:
results_old.tail(5)

,game,s,d,m,hs,as,home,hr,away,ar,csh,csa,combined,venue,links
88,I89,1,6,10,1,0,Forge FC,W,York9 FC,L,1,0,10-6-2019 Forge FC W 1-0 L York9 FC,Tim Hortons Field,https://canpl.ca/matchcentre/5lomje72gq8r3x9xt...
89,I90,1,9,10,1,1,HFX Wanderers FC,D,Pacific FC,D,0,0,10-9-2019 HFX Wanderers FC D 1-1 D Pacific FC,Wanderers Grounds,https://canpl.ca/matchcentre/5m1xy5ksfqgdrzx3g...
90,I91,1,9,10,2,1,Cavalry FC,W,Forge FC,L,0,0,10-9-2019 Cavalry FC W 2-1 L Forge FC,ATCO Field,https://canpl.ca/matchcentre/548tpbwhe2nvkm81p...
91,I92,1,12,10,4,0,York9 FC,W,Forge FC,L,1,0,10-12-2019 York9 FC W 4-0 L Forge FC,York Lions Stadium,https://canpl.ca/matchcentre/5ncl09fzjg81svtrb...
92,I93,1,16,10,1,0,Forge FC,W,Cavalry FC,L,1,0,10-16-2019 Forge FC W 1-0 L Cavalry FC,Tim Hortons Field,https://canpl.ca/matchcentre/5nossz36w45wbtz1z...


In [11]:
results_diff = pd.concat([results, results_old]).drop_duplicates(keep=False)

In [12]:
results_diff

,game,s,d,m,hs,as,home,hr,away,ar,csh,csa,combined,venue,links
93,I94,1,16,10,0,4,Valour FC,L,York9 FC,W,0,1,10-16-2019 Valour FC L 0-4 W York9 FC,IG Field,https://canpl.ca/matchcentre/5oyl3jwgr2padsvpk...
94,I95,1,16,10,3,1,FC Edmonton,W,Pacific FC,L,0,0,10-16-2019 FC Edmonton W 3-1 L Pacific FC,Clarke Stadium,https://canpl.ca/matchcentre/5nw6ub2q95ptd6n71...
95,I96,1,19,10,0,2,York9 FC,L,HFX Wanderers FC,W,0,1,10-19-2019 York9 FC L 0-2 W HFX Wanderers FC,York Lions Stadium,https://canpl.ca/matchcentre/5opkqc01qgmjdcbho...
96,I97,1,19,10,3,1,Cavalry FC,W,FC Edmonton,L,0,0,10-19-2019 Cavalry FC W 3-1 L FC Edmonton,ATCO Field,https://canpl.ca/matchcentre/5nw6ub2q95ptd6n71...
97,I98,1,19,10,2,0,Pacific FC,W,Valour FC,L,1,0,10-19-2019 Pacific FC W 2-0 L Valour FC,Westhills Stadium,https://canpl.ca/matchcentre/5opkqc01qgmjdcbho...
98,I99,2,26,10,1,0,Forge FC,W,Cavalry FC,L,1,0,10-26-2019 Forge FC W 1-0 L Cavalry FC,Tim Hortons Field,https://canpl.ca/matchcentre/7exzoqwijjljpmysg...
99,I100,2,2,11,0,1,Cavalry FC,L,Forge FC,W,0,1,11-2-2019 Cavalry FC L 0-1 W Forge FC,ATCO Field,https://canpl.ca/matchcentre/7f2xl06opfb8koy1j...


In [13]:
def index_reset(data):
    data = data.reset_index()
    data.pop('index')
    return data

def get_team_results(data,query):
    db = data[data['home'] == query]
    da = data[data['away'] == query]
    db = pd.concat([db,da])
    db = index_reset(db)
    return db

def get_team_brief(data,query,df):
    db = get_team_results(data,query)
    cols = ['game','s','csh','csa','combined','venue','links']
    for col in cols:
        db.pop(col)
    db = db.sort_values(by=['m','d'])
    db = index_reset(db)
    db['summary'] = '0'
    for i in range(0,db.shape[0]):
        if db.iloc[i]['home'] == query:
            x = df[df['team'] == db.iloc[i]['away']]#.reset_index()
            opponent = x.iloc[0]['short']
            outcome = db.iloc[i]['hr'] + ' A'
        else:
            x = df[df['team'] == db.iloc[i]['home']]#.reset_index() NOT SURE WHY THIS BROKE SUDDENLY
            opponent = x.iloc[0]['short']
            outcome = db.iloc[i]['ar'] + ' A'
        score = str(db.iloc[i]['hs']) + ' - ' + str(db.iloc[i]['as'])
        db.loc[i,'summary'] = outcome + ' ' + score +  ' ' + opponent
    db['team'] = query
    return db

def get_results_brief(data,dc):
    db = pd.DataFrame()
    for team in dc['team']:
        df = get_team_brief(data,team,dc)
        db = pd.concat([db,df])
    db = index_reset(db)
    return db

def get_club_statistics(data,query):
    df = data
    a = []
    cols = df.columns
    for row in range(0,df.shape[0]):
        if df.iloc[row]['home'] == query:
            if df.iloc[row]['hr'] == 'W':
                vals = [1,3,3,1,0,0,df.iloc[row]['hs'],df.iloc[row]['as'],df.iloc[row]['hs'],df.iloc[row]['as'],0,0]
                a.append(vals)
            if df.iloc[row]['hr'] == 'L':
                vals = [1,0,3,0,1,0,df.iloc[row]['hs'],df.iloc[row]['as'],df.iloc[row]['hs'],df.iloc[row]['as'],0,0]
                a.append(vals)
            if df.iloc[row]['hr'] == 'D':
                vals = [1,1,3,0,0,1,df.iloc[row]['hs'],df.iloc[row]['as'],df.iloc[row]['hs'],df.iloc[row]['as'],0,0]
                a.append(vals)
        if df.iloc[row]['away'] == query:
            if df.iloc[row]['ar'] == 'W':
                vals = [1,3,3,1,0,0,df.iloc[row]['as'],df.iloc[row]['hs'],0,0,df.iloc[row]['as'],df.iloc[row]['hs']]
                a.append(vals)
            if df.iloc[row]['ar'] == 'L':
                vals = [1,0,3,0,1,0,df.iloc[row]['as'],df.iloc[row]['hs'],0,0,df.iloc[row]['as'],df.iloc[row]['hs']]
                a.append(vals)
            if df.iloc[row]['ar'] == 'D':
                vals = [1,1,3,0,0,1,df.iloc[row]['as'],df.iloc[row]['hs'],0,0,df.iloc[row]['as'],df.iloc[row]['hs']]
                a.append(vals)
    db= pd.DataFrame(a,columns=['gp','pts','tpp','w','l','d','gf','ga','gfh','gah','gfa','gaa'])
    db = pd.DataFrame(db.sum())
    db = db.T
    return db

def get_standings(data,season,ref):
    db = pd.DataFrame()
    if season == 1:
        data = data[data['s'] <= 1]
    if season == 2:
        data = data[data['s'] > 1]
    teams = ref['team']
    #teams = np.sort(teams,axis=-1)
    for team in teams:
        df = get_team_brief(data,team,ref)
        df = get_club_statistics(df,team)
        ppg = round(df['pts']/df['gp'],2)
        gd = df['gf'] - df['ga']
        df.insert(0,'team',team)
        df.insert(4,'ppg',ppg)
        df.insert(8,'gd',gd)
        db = pd.concat([db,df])
    db = db.sort_values(by=['pts','w','gf'],ascending=False)
    db = index_reset(db)
    db = db.reset_index()
    db = db.rename(columns={'index':'rank'})
    db['rank'] = db['rank'] + 1
    db = db.fillna(0)
    return db

In [14]:
standings = get_standings(results,1,team_ref)
if results_old.empty == True:
    standings_old = get_standings(results,1,team_ref)
else:
    standings_old = get_standings(results_old,1,team_ref)

In [15]:
standings

,rank,team,gp,pts,tpp,ppg,w,l,d,gd,gf,ga,gfh,gah,gfa,gaa
0,1,Cavalry FC,28,62,84,2.21,19,4,5,32,51,19,26,8,25,11
1,2,Forge FC,28,56,84,2.00,17,6,5,19,45,26,25,10,20,16
2,3,York9 FC,28,34,84,1.21,9,12,7,2,39,37,18,19,21,18
3,4,FC Edmonton,28,32,84,1.14,8,12,8,-6,27,33,15,14,12,19
4,5,Pacific FC,28,31,84,1.11,8,13,7,-11,35,46,22,20,13,26
5,6,Valour FC,28,28,84,1.00,8,16,4,-22,30,52,17,32,13,20
6,7,HFX Wanderers FC,28,28,84,1.00,6,12,10,-14,21,35,13,9,8,26


In [16]:
standings_old

,rank,team,gp,pts,tpp,ppg,w,l,d,gd,gf,ga,gfh,gah,gfa,gaa
0,1,Cavalry FC,27,59,81,2.19,18,4,5,30,48,18,23,7,25,11
1,2,Forge FC,28,56,84,2.00,17,6,5,19,45,26,25,10,20,16
2,3,York9 FC,26,31,78,1.19,8,11,7,0,35,35,18,17,17,18
3,4,FC Edmonton,26,29,78,1.12,7,11,8,-6,23,29,12,13,11,16
4,5,Valour FC,26,28,78,1.08,8,14,4,-16,30,46,17,28,13,18
5,6,Pacific FC,26,28,78,1.08,7,12,7,-11,32,43,20,20,12,23
6,7,HFX Wanderers FC,27,25,81,0.93,5,12,10,-16,19,35,13,9,6,26


In [17]:
def compare_standings(db,df,dc):
    a = []
    for team in dc['team']:
        rank1 = df[df['team'] == team]
        rank2 = db[db['team'] == team]
        if rank1.iloc[0]['rank'] == rank2.iloc[0]['rank']:
            change = 0
        else:
            change = rank1.iloc[0]['rank'] - rank2.iloc[0]['rank']
        a.append([team,change])
    db = pd.DataFrame(a)
    db = pd.DataFrame({'team': db.iloc[:][0], 'change': db.iloc[:][1]})
    db = db.sort_values(by=['change'],ascending=False)
    db = index_reset(db)
    return db

In [18]:
compare_standings_test = compare_standings(standings,standings_old,team_ref)

In [19]:
compare_standings_test

,team,change
0,Pacific FC,1
1,Cavalry FC,0
2,FC Edmonton,0
3,Forge FC,0
4,HFX Wanderers FC,0
5,York9 FC,0
6,Valour FC,-1


In [20]:
def clean_team_game(data,db,check): # Fix this section for teams that haven't played yet
    if check == 0:
        df = data.iloc[0]['team'] # Getting the name of the top team
    else:
        df = data.iloc[-1]['team'] # Getting the name of the bottom placed team
    if data.iloc[-1]['gp'] == 0 and check == 1:
        db = pd.DataFrame([(df,0,df,0)],columns=['home','hs','away','as']) # make an empty set if the game is empty
    else:
        df = db[(db['home'] == df) | (db['away'] == df)] # get appropirate game results for specified team
        db = index_reset(df)
        db = db.iloc[0][['home','hs','away','as']]
        db = pd.DataFrame(db)
        db = db.T
    return db

def get_short_name(data,dc):
    for team in data['home']:
        row = dc[dc['team'] == team]
        data['home'] = row.iloc[0]['short']
    for team in data['away']:
        row = dc[dc['team'] == team]
        data['away'] = row.iloc[0]['short']
    return data

def get_weeks_results(data,standings,dc):
    if data.iloc[0]['hr'] == 'E':
        db = pd.DataFrame([('NA',0,'NA',0)],columns=['home','hs','away','as'])
        big_win, top_team, low_team = db,db,db
        goals = 0
        return db,goals,big_win,top_team,low_team
    df = data
    month = df.iloc[-1]['m']
    week = df.iloc[-1]['d']
    db = df[df['m'] == month]
    db = db[db['d'] >= week - 6]
    db = db.sort_values(by=['game'],ascending=False)
    goals = db['hs'].sum() + db['as'].sum()
    max_home = db[db['hs'] == db['hs'].max()]
    max_away = db[db['as'] == db['as'].max()]
    if max_home.iloc[0]['hs'] > max_away.iloc[0]['as']:
        max_home_win = max_home
    else:
        max_home_win = max_away
    big_win = max_home_win[['home','hs','away','as']]
    big_win = index_reset(big_win)
    big_win = get_short_name(big_win,dc)
    big_win = pd.DataFrame(big_win.loc[0])
    big_win = big_win.T
    top_team = clean_team_game(standings,db,0)
    top_team = get_short_name(top_team,dc)
    low_team = clean_team_game(standings,db,1)
    low_team = get_short_name(low_team,dc)
    return db,goals,big_win,top_team,low_team

In [21]:
game_week, goals, big_win, top_result, low_result = get_weeks_results(results[results['s'] <= 1],standings,team_ref)
game_week

,game,s,d,m,hs,as,home,hr,away,ar,csh,csa,combined,venue,links
97,I98,1,19,10,2,0,Pacific FC,W,Valour FC,L,1,0,10-19-2019 Pacific FC W 2-0 L Valour FC,Westhills Stadium,https://canpl.ca/matchcentre/5opkqc01qgmjdcbho...
96,I97,1,19,10,3,1,Cavalry FC,W,FC Edmonton,L,0,0,10-19-2019 Cavalry FC W 3-1 L FC Edmonton,ATCO Field,https://canpl.ca/matchcentre/5nw6ub2q95ptd6n71...
95,I96,1,19,10,0,2,York9 FC,L,HFX Wanderers FC,W,0,1,10-19-2019 York9 FC L 0-2 W HFX Wanderers FC,York Lions Stadium,https://canpl.ca/matchcentre/5opkqc01qgmjdcbho...
94,I95,1,16,10,3,1,FC Edmonton,W,Pacific FC,L,0,0,10-16-2019 FC Edmonton W 3-1 L Pacific FC,Clarke Stadium,https://canpl.ca/matchcentre/5nw6ub2q95ptd6n71...
93,I94,1,16,10,0,4,Valour FC,L,York9 FC,W,0,1,10-16-2019 Valour FC L 0-4 W York9 FC,IG Field,https://canpl.ca/matchcentre/5oyl3jwgr2padsvpk...
92,I93,1,16,10,1,0,Forge FC,W,Cavalry FC,L,1,0,10-16-2019 Forge FC W 1-0 L Cavalry FC,Tim Hortons Field,https://canpl.ca/matchcentre/5nossz36w45wbtz1z...


In [22]:
big_win

,home,hs,away,as
0,VFC,0,Y9,4


In [23]:
top_result

,home,hs,away,as
0,CFC,3,FCE,1


In [24]:
low_result

,home,hs,away,as
0,Y9,0,HFX,2


In [25]:
goals

17

In [26]:
compare_standings_test.iloc[0]['team']

'Pacific FC'

In [27]:
compare_standings_test.iloc[-1]['team']

'Valour FC'

In [28]:
standings.iloc[0]['team']

'Cavalry FC'

In [29]:
#championship = get_standings(results,2)
#championship = championship[championship['gp'] > 1]
#championship

In [30]:
#championship.to_csv(f'datasets/{year}/cpl-{year}-championship.csv',index=False)

In [31]:
#standings.to_csv(f'datasets/{year}/cpl-{year}-standings.csv',index=False)

In [32]:
results_brief = get_results_brief(results,team_ref)

In [33]:
results_brief.head(5)

,d,m,hs,as,home,hr,away,ar,summary,team
0,4,5,2,1,Cavalry FC,W,York9 FC,L,W A 2 - 1 Y9,Cavalry FC
1,8,5,1,0,Cavalry FC,W,Valour FC,L,W A 1 - 0 VFC,Cavalry FC
2,12,5,1,2,Forge FC,L,Cavalry FC,W,W A 1 - 2 FFC,Cavalry FC
3,18,5,1,0,Cavalry FC,W,FC Edmonton,L,W A 1 - 0 FCE,Cavalry FC
4,25,5,2,0,Cavalry FC,W,HFX Wanderers FC,L,W A 2 - 0 HFX,Cavalry FC


In [34]:
#results_brief.to_csv(f'datasets/{year}/cpl-{year}-results_brief.csv',index=False)

In [35]:
stats.head(5)

,game,team,position,number,name,first,last,minutes,touches,passes,...,clearances,interceptions,yellow,red,f-won,f-conceded,shots faced,saves,claimed crosses,cs
0,I1,Forge FC,m,1,Alexander Achinioti-Jönsson,Alexander,Achinioti-Jönsson,77.0,47.0,41.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,I1,Forge FC,d,16,Bertrand Owundi,Bertrand,Owundi,77.0,69.0,56.0,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,I1,Forge FC,f,13,Christopher Nanco,Christopher,Nanco,56.0,42.0,27.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,I1,Forge FC,d,22,Dominic Samuel,Dominic,Samuel,90.0,59.0,48.0,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,I1,Forge FC,m,6,Elimane Cissé,Elimane,Cissé,90.0,75.0,61.0,...,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [36]:
stats.describe()

,number,minutes,touches,passes,pass-acc,crosses,c-acc,assists,chances,goals,...,clearances,interceptions,yellow,red,f-won,f-conceded,shots faced,saves,claimed crosses,cs
count,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,...,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000
mean,13.829250,71.540768,44.576965,30.244241,0.736570,1.116271,0.093119,0.060695,0.628154,0.087751,...,1.210238,0.790128,0.112249,0.006216,0.938574,0.995978,0.318099,0.227422,0.036563,0.019744
std,11.355872,28.545983,23.277512,18.645196,0.233204,1.904354,0.235873,0.246352,0.989120,0.311297,...,1.859761,1.131050,0.315730,0.078609,1.119698,1.125445,1.295603,0.956352,0.251072,0.139145
min,1.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,61.000000,28.000000,16.000000,0.670000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,90.000000,45.000000,29.000000,0.780000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,19.000000,90.000000,61.000000,42.000000,0.850000,2.000000,0.000000,0.000000,1.000000,0.000000,...,2.000000,1.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,92.000000,90.000000,118.000000,112.000000,1.000000,15.000000,1.000000,2.000000,7.000000,3.000000,...,11.000000,7.000000,1.000000,1.000000,6.000000,7.000000,13.000000,10.000000,4.000000,1.000000


In [42]:
def get_team_stats(data,query):
    db = data[data['team'] == query]
    names = db['name'].unique()
    information = data.copy()
    db.pop('number')
    db = db.groupby(['name']).sum()
    db.insert(0,'last','empty')
    db.insert(0,'first','empty')
    db.insert(0,'position','empty')
    db.insert(0,'number',0)
    #db.insert(0,'team',team)
    i = 0
    for name in names:
        player = information[information['name'] == name].head(1)
        db.at[name,'first'] = player.iloc[0]['first']
        db.at[name,'last'] = player.iloc[0]['last']
        db.at[name,'number'] = int(player.iloc[0]['number'])
        db.at[name,'position'] = player.iloc[0]['position']
        db.at[name,'pass-acc'] = player.iloc[0]['pass-acc'].mean()
        db.at[name,'cross-acc'] = player.iloc[0]['cross-acc'].mean()
    db = db.reset_index()
    return db

def get_stats_all(data,dc):
    db = pd.DataFrame()
    for team in dc['team']:
        df = get_team_stats(data,team)
        df.insert(0,'team',team)
        db = pd.concat([db,df])
    db = index_reset(db)
    return db

In [43]:
team_stats = get_stats_all(stats,team_ref)

In [44]:
team_stats

,team,name,number,position,first,last,minutes,touches,passes,pass-acc,...,clearances,interceptions,yellow,red,f-won,f-conceded,shots faced,saves,claimed crosses,cs
0,Cavalry FC,Aribim Pepple,24,f,Aribim,Pepple,60.0,21.0,9.0,1.00,...,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0
1,Cavalry FC,Carlos Patiño,20,m,Carlos,Patiño,320.0,238.0,122.0,0.88,...,1.0,5.0,0.0,0.0,13.0,12.0,0.0,0.0,0.0,0.0
2,Cavalry FC,Dean Northover,12,d,Dean,Northover,609.0,478.0,255.0,0.67,...,10.0,15.0,2.0,1.0,14.0,13.0,0.0,0.0,0.0,0.0
3,Cavalry FC,Dominick Zator,4,d,Dominick,Zator,2335.0,1866.0,1402.0,0.83,...,75.0,51.0,0.0,0.0,19.0,15.0,0.0,0.0,0.0,0.0
4,Cavalry FC,Dominique Malonga,19,f,Dominique,Malonga,1871.0,760.0,489.0,0.80,...,12.0,2.0,1.0,0.0,31.0,8.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,York9 FC,Ryan Telfer,18,m,Ryan,Telfer,2216.0,1533.0,832.0,0.60,...,8.0,23.0,2.0,0.0,45.0,25.0,0.0,0.0,0.0,0.0
160,York9 FC,Simon Karlsson Adjei,12,f,Simon,Karlsson Adjei,1422.0,575.0,341.0,0.88,...,6.0,2.0,0.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0
161,York9 FC,Stefan Lamanna,21,f,Stefan,Lamanna,77.0,55.0,31.0,0.86,...,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
162,York9 FC,Steven Furlano,77,d,Steven,Furlano,517.0,325.0,225.0,0.85,...,6.0,10.0,4.0,0.0,4.0,15.0,0.0,0.0,0.0,0.0


In [70]:
team_stats.to_csv(f'datasets/{year}/cpl-{year}-team_stats.csv',index=False)

In [48]:
# get associated information for players league wide and calculate an overall score for each position
def get_evaluation(condensed_player_info,full_player_info):
    names = condensed_player_info.name.unique() # grab the list of names at the specified position
    eval_ = condensed_player_info.describe().T # get the evalution scores
    checks = condensed_player_info.columns[4:] # slice away the first three columns (name,number,postion) not needed
    condensed_player_info['overall'] = 0.0 # create the final column overall
    condensed_player_info = condensed_player_info.set_index('name') # set the index to the player name to search for a specific player
    for name in names: # iterate through the names in the lisst
        player = full_player_info[full_player_info['name'] == name].head(1) # get the players details
        a = [] # create an empty array to store the scores
        for check in checks: # iterate through the columns of remaining data
            result = player.iloc[0][check] / eval_['max'][check] # calculate the score for the value found value/max
            a.append(result) # append the result into the list
            overall = round(sum(a) / len(checks),2) #calculate the final score sum(list) / num of checks
            condensed_player_info.at[name,'overall'] = overall # assign the value as the overall score
    condensed_player_info = condensed_player_info.reset_index() # reset the index, making the name column a column again
    condensed_player_info = condensed_player_info.sort_values(by=['overall'],ascending=False) # sort using overall, descending
    return condensed_player_info

In [49]:
def top_tracked(team_stats,tracked):
    if team_stats.minutes.sum() == 0:
        tracked_player_stat = pd.team_statsFrame([('NA',0,0,0,0)],columns=['team','name','number','minutes','goals'])
        return tracked_player_stat
    df = team_stats.copy()
    cols = ['team','name','position','number','minutes',tracked]
    tracked_player_stat = df[cols]
    #tracked_player_stat = get_evaluation(tracked_player_stat,df)
    tracked_player_stat = tracked_player_stat.sort_values(by=[tracked],ascending=False)
    tracked_player_stat = tracked_player_stat.reset_index()
    tracked_player_stat.pop('index')
    team = tracked_player_stat.pop('team')
    tracked_player_stat.insert(0,'team',team)
    tracked_player_stat = tracked_player_stat[tracked_player_stat[tracked] >= 1]
    rank = tracked_player_stat.index + 1
    tracked_player_stat.insert(0,'rank',rank)
    return tracked_player_stat

In [50]:
def top_position(team_stats,position): # get the forwards in the league
    if team_stats.minutes.sum() == 0:
        if position == 'f':
            condensed_player_info = pd.team_statsFrame([('NA',0,0,0,0,0,0,0,0,0,0,0,0,0)],columns=['team','name','number','position','minutes','goals','chances','assists','shots','s-target','passes','crosses','duels','tackles'])
        if position == 'm':
            condensed_player_info = pd.team_statsFrame([('NA',0,0,0,0,0,0,0,0,0,0,0,0,0,0)],columns=['team','name','number','position','minutes','goals','assists','touches','passes','pass-acc','crosses','cross-acc','chances','duels','tackles'])
        if position == 'd':
            condensed_player_info = pd.team_statsFrame([('NA',0,0,0,0,0,0,0,0,0,0)],columns=['team','name','number','position','minutes','tackles','t-won','clearances','interceptions','duels','d-won'])
        if position == 'g':
            condensed_player_info = pd.team_statsFrame([('NA',0,0,0,0,0,0,0,0)],columns=['team','name','number','position','minutes','cs','saves','shots faced','claimed crosses'])
        condensed_player_info = pd.team_statsFrame([('NA',0,0,0,0,0,0,0,0,0,0,0,0,0)],columns=['team','name','number','position','minutes','goals','chances','assists','shots','shots on target','passes','crosses','duels','tackles'])
        return condensed_player_info
    player_information = team_stats.copy() # load player information
    if position == 'f':
        cols = ['team','name','number','position','minutes','goals','chances','assists','shots','s-target','passes','crosses','duels','tackles']
    if position == 'm':
        cols = ['team','name','number','position','minutes','goals','assists','touches','passes','pass-acc','crosses','cross-acc','chances','duels','tackles']
    if position == 'd':
        cols = ['team','name','number','position','minutes','tackles','t-won','clearances','interceptions','duels','d-won']
    if position == 'g':
        cols = ['team','name','number','position','minutes','cs','saves','shots faced','claimed crosses']
    full_player_info = player_information[player_information['position'] == position] # get the forwards where position = f
    condensed_player_info = full_player_info[cols] # select specific columns associated with the evaluation
    condensed_player_info = get_evaluation(condensed_player_info,full_player_info) # condensed Dataframe and full Dataframe being passes
    condensed_player_info = index_reset(condensed_player_info)
    names = condensed_player_info.name.unique() # get the names of the players who fit the criteria
    condensed_player_info = condensed_player_info.set_index('name') # set the index to the name column to make the search possible
    if position == 'f':
        for name in names:
            player = full_player_info[full_player_info['name'] == name].head(1) # forwards main purpose is to score goals
            if (player.iloc[0]['goals'] <= 2.0 and player.iloc[0]['minutes'] >= 1000.0): # if player scores less than 2 & has minutes greater than 1000
                condensed_player_info.at[name,'overall'] = condensed_player_info.at[name,'overall'] - 0.1
            if player.iloc[0]['goals'] >= 8.0: # reward forwards scoring greater than 8 goals
                condensed_player_info.at[name,'overall'] = condensed_player_info.at[name,'overall'] + 0.15
    condensed_player_info = condensed_player_info.sort_values(by=['overall'],ascending=False)
    condensed_player_info = condensed_player_info.reset_index()
    team = condensed_player_info.pop('team')
    condensed_player_info.insert(0,'team',team)
    return condensed_player_info

In [59]:
def top_offenders(data):  # get the offences handed out in the league
    if data.minutes.sum() == 0:
        db = pd.DataFrame([('NA',0,0,0,0,0,0)],columns=['team','name','number','minutes','yellow','red','f-conceded'])
        return db
    player_information = data.copy()
    cols = ['team','name','position','number','minutes','yellow','red','f-conceded']
    df = player_information
    db = df[cols]
    db = get_evaluation(db,df)
    db = db.sort_values(by=['red','yellow'],ascending=False)
    db = db.reset_index()
    db.pop('index')
    team = db.pop('team')
    db.insert(0,'team',team)
    return db

In [60]:
rated_goalscorers = top_tracked(team_stats,'goals')
rated_assists = top_tracked(team_stats,'assists')
rated_assists.head(10)

,rank,team,name,position,number,minutes,assists
0,1,Pacific FC,Blake Smith,d,4,2004.0,5.0
1,2,Valour FC,Michael Petrasso,m,9,1301.0,5.0
2,3,Pacific FC,Ben Fisk,m,21,1662.0,5.0
3,4,Forge FC,Tristan Borges,m,4,1884.0,5.0
4,5,Forge FC,Kyle Bekker,m,3,2354.0,5.0
5,6,Forge FC,Kwame Awuah,d,2,1885.0,5.0
6,7,York9 FC,Diyaeddine Abzi,d,20,1814.0,4.0
7,8,York9 FC,Morey Doner,d,3,2250.0,4.0
8,9,York9 FC,Kyle Porter,m,19,1916.0,4.0
9,10,Cavalry FC,Elijah Adekugbe,m,16,1575.0,4.0


In [61]:
rated_goalscorers.head(10)

,rank,team,name,position,number,minutes,goals
0,1,Forge FC,Tristan Borges,m,4,1884.0,13.0
1,2,Cavalry FC,Dominique Malonga,f,19,1871.0,11.0
2,3,Pacific FC,Terran Campbell,f,14,2334.0,11.0
3,4,FC Edmonton,Easton Ongaro,f,19,1167.0,10.0
4,5,York9 FC,Rodrigo Gattas,f,22,1958.0,9.0
5,6,York9 FC,Ryan Telfer,m,18,2216.0,8.0
6,7,Cavalry FC,Oliver Minatel,f,7,1450.0,7.0
7,8,Valour FC,Marco Bustos,m,22,2249.0,7.0
8,9,York9 FC,Simon Karlsson Adjei,f,12,1422.0,7.0
9,10,HFX Wanderers FC,Akeem Garcia,m,11,2082.0,7.0


In [62]:
rated_forwards = top_position(team_stats,'f')
rated_midfielders = top_position(team_stats,'m')
rated_defenders = top_position(team_stats,'d')
rated_keepers = top_position(team_stats,'g')
rated_offenders = top_offenders(team_stats)

/home/todd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
rated_forwards.to_csv(f'datasets/{year}/cpl-{year}-forwards.csv',index=False)
rated_midfielders.to_csv(f'datasets/{year}/cpl-{year}-midfielders.csv',index=False)
rated_defenders.to_csv(f'datasets/{year}/cpl-{year}-defenders.csv',index=False)
rated_keepers.to_csv(f'datasets/{year}/cpl-{year}-keepers.csv',index=False)
rated_offenders.to_csv(f'datasets/{year}/cpl-{year}-discipline.csv',index=False)

In [64]:
rated_forwards.head(10)

,team,name,number,position,minutes,goals,chances,assists,shots,s-target,passes,crosses,duels,tackles,overall
0,Pacific FC,Terran Campbell,14,f,2334.0,11.0,31.0,2.0,53.0,23.0,569.0,29.0,324.0,39.0,0.85
1,York9 FC,Rodrigo Gattas,22,f,1958.0,9.0,23.0,0.0,77.0,36.0,728.0,38.0,214.0,17.0,0.76
2,Cavalry FC,Dominique Malonga,19,f,1871.0,11.0,19.0,1.0,68.0,34.0,489.0,5.0,185.0,2.0,0.70
3,HFX Wanderers FC,Mohamed Kourouma,12,f,1988.0,1.0,44.0,3.0,68.0,17.0,702.0,156.0,264.0,56.0,0.67
4,Cavalry FC,Nico Pasquotti,17,f,1853.0,5.0,24.0,3.0,33.0,16.0,457.0,61.0,374.0,47.0,0.64
5,FC Edmonton,Oumar Diouck,45,f,1977.0,6.0,30.0,2.0,45.0,16.0,583.0,53.0,269.0,36.0,0.61
6,FC Edmonton,Easton Ongaro,19,f,1167.0,10.0,11.0,2.0,38.0,20.0,249.0,6.0,188.0,5.0,0.58
7,York9 FC,Simon Karlsson Adjei,12,f,1422.0,7.0,21.0,2.0,53.0,16.0,341.0,4.0,210.0,15.0,0.47
8,Forge FC,Christopher Nanco,13,f,1429.0,4.0,21.0,2.0,29.0,12.0,380.0,22.0,208.0,19.0,0.43
9,Cavalry FC,Oliver Minatel,7,f,1450.0,7.0,17.0,1.0,37.0,12.0,348.0,21.0,194.0,29.0,0.43


In [65]:
rated_midfielders.head(10)

,team,name,number,position,minutes,goals,assists,touches,passes,pass-acc,crosses,cross-acc,chances,duels,tackles,overall
0,Forge FC,Tristan Borges,4,m,1884.0,13.0,5.0,1307.0,680.0,0.74,77.0,0.00,42.0,356.0,59.0,0.71
1,York9 FC,Ryan Telfer,18,m,2216.0,8.0,1.0,1533.0,832.0,0.60,164.0,0.00,66.0,372.0,52.0,0.70
2,Cavalry FC,Julian Büscher,8,m,1678.0,5.0,4.0,1432.0,971.0,0.80,113.0,0.29,38.0,282.0,57.0,0.68
3,Forge FC,Kyle Bekker,3,m,2354.0,4.0,5.0,1748.0,1362.0,0.82,98.0,0.25,44.0,173.0,23.0,0.68
4,York9 FC,Manuel Aparicio,10,m,2159.0,2.0,2.0,1788.0,1273.0,0.67,40.0,0.33,37.0,353.0,56.0,0.64
5,Valour FC,Marco Bustos,22,m,2249.0,7.0,3.0,1417.0,912.0,0.93,74.0,0.00,43.0,299.0,22.0,0.61
6,Valour FC,Louis Béland-Goyette,5,m,1772.0,1.0,3.0,1160.0,882.0,0.65,10.0,0.67,28.0,217.0,54.0,0.55
7,York9 FC,Kyle Porter,19,m,1916.0,2.0,4.0,1131.0,729.0,0.76,88.0,0.00,31.0,216.0,47.0,0.55
8,Cavalry FC,José Escalante,11,m,1592.0,4.0,4.0,905.0,450.0,0.68,116.0,0.25,29.0,226.0,32.0,0.53
9,Pacific FC,Victor Blasco,19,m,1586.0,6.0,3.0,1040.0,589.0,0.67,47.0,0.00,27.0,349.0,43.0,0.52


In [66]:
rated_defenders.head(10)

,team,name,number,position,minutes,tackles,t-won,clearances,interceptions,duels,d-won,overall
0,FC Edmonton,Mélé Temguia,8,d,2430.0,61.0,41.0,82.0,51.0,244.0,140.0,0.87
1,HFX Wanderers FC,André Bona,3,d,2053.0,66.0,50.0,76.0,32.0,257.0,150.0,0.84
2,York9 FC,Luca Gasparotto,13,d,2520.0,31.0,24.0,117.0,49.0,254.0,182.0,0.83
3,Cavalry FC,Dominick Zator,4,d,2335.0,46.0,32.0,75.0,51.0,265.0,169.0,0.83
4,York9 FC,Diyaeddine Abzi,20,d,1814.0,48.0,35.0,31.0,48.0,267.0,134.0,0.72
5,Forge FC,Kwame Awuah,2,d,1885.0,57.0,40.0,17.0,39.0,258.0,124.0,0.71
6,Forge FC,Daniel Krutzen,10,d,2381.0,48.0,24.0,85.0,42.0,186.0,110.0,0.71
7,Pacific FC,Kadin Chung,7,d,2032.0,57.0,42.0,42.0,31.0,203.0,111.0,0.69
8,HFX Wanderers FC,Matthew Arnone,23,d,2094.0,31.0,20.0,99.0,49.0,170.0,106.0,0.67
9,HFX Wanderers FC,Ndzemdzela Langwa,20,d,1281.0,65.0,47.0,20.0,14.0,272.0,154.0,0.67


In [67]:
rated_keepers.head(10)

,team,name,number,position,minutes,cs,saves,shots faced,claimed crosses,overall
0,York9 FC,Nathan Ingham,1,g,2340.0,6.0,88.0,122.0,17.0,0.93
1,Cavalry FC,Marco Carducci,1,g,2160.0,9.0,65.0,81.0,11.0,0.79
2,Forge FC,Triston Henry,9,g,1890.0,8.0,53.0,70.0,12.0,0.72
3,FC Edmonton,Connor James,1,g,1980.0,6.0,67.0,91.0,9.0,0.71
4,Pacific FC,Mark Village,1,g,1440.0,4.0,47.0,69.0,10.0,0.55
5,Valour FC,Tyson Farago,1,g,1440.0,2.0,47.0,77.0,13.0,0.55
6,HFX Wanderers FC,Christian Oxner,50,g,1489.0,5.0,41.0,56.0,10.0,0.54
7,Pacific FC,Nolan Wirth,6,g,1080.0,1.0,59.0,83.0,6.0,0.46
8,Valour FC,Mathias Janssens,26,g,1080.0,4.0,43.0,65.0,0.0,0.39
9,HFX Wanderers FC,Jan-Michael Williams,21,g,1028.0,2.0,40.0,59.0,4.0,0.37


In [68]:
rated_offenders.head(20)

,team,name,position,number,minutes,yellow,red,f-conceded,overall
0,York9 FC,Manuel Aparicio,m,10,2159.0,5.0,1.0,56.0,0.83
1,Forge FC,Dominic Samuel,d,22,2044.0,5.0,1.0,38.0,0.74
2,Pacific FC,Matthew Baldisimo,m,17,1809.0,5.0,1.0,34.0,0.71
3,Pacific FC,Alexander González,m,28,1746.0,4.0,1.0,34.0,0.67
4,Valour FC,Jordan Murrell,d,4,1784.0,4.0,1.0,30.0,0.66
5,Cavalry FC,Joel Waterman,m,15,1719.0,4.0,1.0,20.0,0.61
6,Valour FC,Adam Mitter,d,15,1513.0,4.0,1.0,23.0,0.60
7,Cavalry FC,Nathan Mavila,d,3,1952.0,2.0,1.0,38.0,0.65
8,Forge FC,Tristan Borges,m,4,1884.0,2.0,1.0,37.0,0.64
9,Pacific FC,Lukas MacNaughton,d,14,1704.0,2.0,1.0,24.0,0.57


In [58]:
def get_match_tables(data,query):
    db = data[data['home'] == query]
    db = pd.concat([db,data[data['away'] == query]])
    db = db.sort_values(by=['m','d'])
    return db

def likelihood_input(array,a_list):
    b = a_list[0]
    c = a_list[1]
    d = a_list[2]
    array.append(b)
    array.append(c)
    array.append(d)
    return array

def likelihood_table(data,query):
    df = get_match_tables(data,query)
    array = []
    cols = data.columns
    for row in range(0,df.shape[0]):
        if df.iloc[row]['home'] == query:
            if df.iloc[row]['hr'] == 'W':
                array = likelihood_input(array,[[1,2,1],[1,0,0],[1,1,0]])
            if df.iloc[row]['hr'] == 'L':
                array = likelihood_input(array,[[1,2,0],[1,0,1],[1,1,0]])
            if df.iloc[row]['hr'] == 'D':
                array = likelihood_input(array,[[1,2,0],[1,0,0],[1,1,1]])
        if df.iloc[row]['away'] == query:
            if df.iloc[row]['ar'] == 'W':
                array = likelihood_input(array,[[2,2,1],[2,0,0],[2,1,0]])
            if df.iloc[row]['ar'] == 'L':
                array = likelihood_input(array,[[2,2,0],[2,0,1],[2,1,0]])
            if df.iloc[row]['ar'] == 'D':
                array = likelihood_input(array,[[2,2,1],[2,0,0],[2,1,1]])
    db= pd.DataFrame(array,columns=['h/a','w/l/d','y/n'])
    return db

def get_team_comparison(data,q1,q2):
    # getting games with q1 in both home or away
    db = data[data['team'] == q1]
    db = db.reset_index()
    db.pop('index')
    # filering down more to get only the games against q2
    db = db.sort_values(by=['m','d'])
    db = db[(db['home'] == q2) | (db['away'] == q2)]
    db = db.reset_index()
    db.pop('index')
    if db.empty == True:
        db = pd.DataFrame([(0,0,0,0,q1,'D',q2,'D','empty',q1)],columns=['d','m','hs','as','home','hr','away','ar','summary','team'])
    return db

def get_NB_data(data,query):
    db = likelihood_table(data,query)
    dy = db.pop('y/n').to_list()
    dx = [tuple(x) for x in db.values]
    return dx, dy

In [59]:
schedule = get_schedule(results)
schedule

,game,home,away
0,I1,Forge FC,York9 FC
9,I10,Forge FC,Cavalry FC
10,I11,FC Edmonton,Pacific FC
11,I12,Valour FC,Forge FC
12,I13,Pacific FC,York9 FC
...,...,...,...
93,I94,Valour FC,York9 FC
94,I95,FC Edmonton,Pacific FC
95,I96,York9 FC,HFX Wanderers FC
96,I97,Cavalry FC,FC Edmonton


In [62]:
#schedule.to_csv(f'datasets/{year}/cpl-{year}-schedule.csv',index=False)

In [63]:
schedule.iloc[0]['home']

'Forge FC'

In [64]:
# home side
q1 = schedule.iloc[0]['home']
# away side
q2 = schedule.iloc[0]['away']
print(q1,q2)

Forge FC York9 FC


In [65]:
compare = get_team_comparison(results_brief,q1,q2)

In [66]:
compare

,d,m,hs,as,home,hr,away,ar,summary,team
0,27,4,1,1,Forge FC,D,York9 FC,D,D A 1 - 1 Y9,Forge FC
1,25,5,0,2,York9 FC,L,Forge FC,W,W A 0 - 2 Y9,Forge FC
2,8,9,2,1,Forge FC,W,York9 FC,L,W A 2 - 1 Y9,Forge FC
3,6,10,1,0,Forge FC,W,York9 FC,L,W A 1 - 0 Y9,Forge FC
4,12,10,4,0,York9 FC,W,Forge FC,L,L A 4 - 0 Y9,Forge FC


In [67]:
t1_x, t1_y = get_NB_data(compare,q1)
t2_x, t2_y = get_NB_data(compare,q2)

### Game Prediction

use Gaussian Naive Bayes model to predict the outcome of a home match.

In [68]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB,BernoulliNB
import statistics

In [69]:
def get_gnb_prediction(query,x,y,result):
    
    gnb = GaussianNB()
    bnb = BernoulliNB()
    # Train the model using the training sets
    
    gnb.fit(x,y)
    bnb.fit(x,y)
    
    # use below instead of predicted = model.predict([result]) because we want the probability
    gnb_pred = np.round(gnb.predict_proba([result])[:, 1],decimals=2)
    bnb_pred = np.round(bnb.predict_proba([result])[:, 1],decimals=2)
    
    pred = round((gnb_pred[0] + bnb_pred[0]) / 2,2)
    #print(gnb_pred[0], bnb_pred[0], pred)
    
    return pred

def get_match_prediction_home(query,x,y):
    home_win = get_gnb_prediction(query,x,y,[1,2])
    draw = get_gnb_prediction(query,x,y,[1,1])
    return home_win, draw
    
def get_match_prediction_away(query,x,y):
    away_win = get_gnb_prediction(query,x,y,[2,2])
    return away_win

def get_match_prediction(q1,q2,x1,y1,x2,y2):
    if len(x1) == 0:
        x = round(1/3,2)
        home_win, away_win,draw = x,x,x
        return home_win,away_win,draw
    home_win, draw = get_match_prediction_home(q1,x1,y1)
    away_win = get_match_prediction_away(q2,x2,y2)
    return home_win, draw, away_win

In [70]:
home_win, draw, away_win = get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)

In [71]:
print(q1,'\nwin probability: ', round(home_win,2))

Forge FC 
win probability:  0.46


In [72]:
print(q2,'\nwin probability: ', round(away_win,2))

York9 FC 
win probability:  0.34


In [73]:
print('Draw probability: ', round(draw,2))

Draw probability:  0.33


In [74]:
round(home_win + draw + away_win,1)

1.1

In [70]:
def get_team_form(data,query):
    db = data[data['team'] == query]
    db = pd.DataFrame(db['summary'])
    return db

def get_form_results(data,dc):
    db = pd.DataFrame()
    form = get_results_brief(data[data['s'] <= 1],dc)
    teams = data.home.unique()
    teams = np.sort(teams,axis=-1)
    for team in teams:
        df = get_team_form(form,team)
        #print(team,'\n',df)
        db[team] = pd.Series(df['summary'].values)
    db = db.T
    db = db.reset_index()
    db = db.fillna('E')
    return db

In [71]:
team_form_results = get_form_results(results,team_ref)
team_form_results

,index,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,Cavalry FC,W A 2 - 1 Y9,W A 1 - 0 VFC,W A 1 - 2 FFC,W A 1 - 0 FCE,W A 2 - 0 HFX,W A 0 - 3 FCE,W A 1 - 2 HFX,L A 0 - 1 FFC,W A 0 - 2 Y9,...,D A 1 - 1 PFC,W A 0 - 8 VFC,W A 0 - 1 FCE,W A 4 - 1 PFC,D A 1 - 1 Y9,W A 4 - 1 VFC,W A 2 - 0 HFX,W A 2 - 1 FFC,L A 1 - 0 FFC,W A 3 - 1 FCE
1,FC Edmonton,W A 1 - 2 VFC,D A 0 - 0 PFC,L A 1 - 0 CFC,L A 2 - 0 FFC,L A 0 - 1 VFC,L A 0 - 3 CFC,D A 0 - 0 Y9,W A 1 - 3 PFC,W A 1 - 0 FFC,...,D A 2 - 2 Y9,D A 0 - 0 HFX,L A 0 - 1 CFC,L A 1 - 0 PFC,L A 1 - 3 Y9,D A 1 - 1 HFX,L A 0 - 1 FFC,L A 3 - 1 VFC,W A 3 - 1 PFC,L A 3 - 1 CFC
2,Forge FC,D A 1 - 1 Y9,L A 2 - 1 HFX,W A 3 - 0 PFC,L A 1 - 2 CFC,W A 0 - 2 VFC,W A 0 - 2 Y9,W A 2 - 0 FCE,W A 2 - 1 VFC,W A 0 - 1 CFC,...,W A 2 - 1 Y9,W A 1 - 3 VFC,D A 2 - 2 HFX,D A 1 - 1 HFX,W A 3 - 0 PFC,W A 0 - 1 FCE,W A 1 - 0 Y9,L A 2 - 1 CFC,L A 4 - 0 Y9,W A 1 - 0 CFC
3,HFX Wanderers FC,L A 1 - 0 PFC,W A 2 - 1 FFC,L A 1 - 0 VFC,L A 2 - 0 CFC,D A 1 - 1 Y9,W A 2 - 1 PFC,L A 1 - 2 CFC,D A 0 - 0 Y9,W A 2 - 0 VFC,...,L A 2 - 0 VFC,D A 0 - 0 FCE,D A 2 - 2 FFC,D A 1 - 1 FFC,D A 0 - 0 VFC,D A 1 - 1 FCE,D A 1 - 1 PFC,L A 2 - 0 CFC,D A 1 - 1 PFC,W A 0 - 2 Y9
4,Pacific FC,W A 1 - 0 HFX,L A 1 - 2 VFC,L A 3 - 0 FFC,D A 0 - 0 FCE,D A 2 - 2 Y9,L A 2 - 1 HFX,L A 1 - 0 Y9,W A 1 - 2 VFC,L A 1 - 3 FCE,...,D A 1 - 1 CFC,D A 1 - 1 FFC,L A 0 - 2 Y9,W A 1 - 0 FCE,L A 4 - 1 CFC,L A 3 - 0 FFC,D A 1 - 1 HFX,D A 1 - 1 HFX,L A 3 - 1 FCE,W A 2 - 0 VFC
5,Valour FC,W A 1 - 2 PFC,L A 1 - 2 FCE,L A 1 - 0 CFC,W A 1 - 0 HFX,L A 0 - 2 FFC,W A 0 - 1 FCE,L A 2 - 1 FFC,L A 1 - 2 PFC,L A 2 - 0 HFX,...,L A 2 - 1 PFC,W A 2 - 0 HFX,L A 0 - 8 CFC,L A 1 - 3 FFC,W A 2 - 4 Y9,D A 0 - 0 HFX,L A 4 - 1 CFC,W A 3 - 1 FCE,L A 0 - 4 Y9,L A 2 - 0 PFC
6,York9 FC,D A 1 - 1 FFC,L A 2 - 1 CFC,D A 2 - 2 PFC,L A 0 - 2 FFC,D A 1 - 1 HFX,W A 1 - 0 PFC,D A 0 - 0 FCE,D A 0 - 0 HFX,L A 0 - 2 CFC,...,D A 2 - 2 FCE,L A 2 - 1 FFC,W A 0 - 2 PFC,L A 2 - 4 VFC,W A 1 - 3 FCE,D A 1 - 1 CFC,L A 1 - 0 FFC,W A 4 - 0 FFC,W A 0 - 4 VFC,L A 0 - 2 HFX


In [72]:
team_form_results.to_csv(f'datasets/{year}/cpl-{year}-team_form.csv',index=False)

In [73]:
def get_roster(query,stats,team_ref): # use team stats to get the player information
    roster = get_stats_all(stats,team_ref)
    roster = roster[roster['team'] == query]
    roster = roster[['name','number','position']]
    roster.insert(3,'overall',0)
    roster = index_reset(roster)
    return roster

def get_home_away_comparison(stats,game,team):
    db = stats[stats['game'] == game].copy()
    db = db[db['team'] == team]
    db = db.sort_values(by=['minutes'],ascending=False)
    db = db#[0:11]
    db = db['name']
    return db

In [74]:
def get_compare_roster(results,query,stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers):
    # going through the rated players to get the best players for each position
    # using game_h,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,results,team_stats
    roster = get_roster(query,stats,team_ref)
    keepers = [] # lists for each position
    forwards = []
    midfields = []
    defenders = []
    for name in roster['name']:
        player = roster[roster['name'] == name]
        if player.iloc[0][2] == 'f':
            player = rated_forwards[rated_forwards['name'] == name]
            player = player.iloc[0].copy()
            player['asc'] = 0
            forwards.append(player)
        if player.iloc[0][2] == 'm':
            player = rated_midfielders[rated_midfielders['name'] == name]
            player = player.iloc[0].copy()
            player['asc'] = 1
            midfields.append(player)
        if player.iloc[0][2] == 'd':
            player = rated_defenders[rated_defenders['name'] == name]
            player = player.iloc[0].copy()
            player['asc'] = 2
            defenders.append(player)
        if player.iloc[0][2] == 'g':
            player = rated_keepers[rated_keepers['name'] == name]
            player = player.iloc[0].copy()
            player['asc'] = 3
            keepers.append(player)
    db = pd.DataFrame(keepers)
    db = db.sort_values(by=['asc','overall'],ascending=False)
    dd = pd.DataFrame(defenders)
    dd = dd.sort_values(by=['asc','overall'],ascending=False)
    dm = pd.DataFrame(midfields)
    dm = dm.sort_values(by=['asc','overall'],ascending=False)
    df = pd.DataFrame(forwards)
    df = df.sort_values(by=['asc','overall'],ascending=False)
    db = pd.concat([db[0:1],dd[0:4],dm[0:4],df[0:2]])
    db = db[['name','number','position','overall','asc']]
    if results.iloc[0]['hr'] == 'E': # check if games haven't been played
        db = db.sort_values(by=['asc'],ascending=False)
    else:
        db = db.sort_values(by=['asc','overall'],ascending=False)
    db = index_reset(db)
    db.pop('asc')
    return db

In [75]:
h1_roster = get_compare_roster(results,q1,team_stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers)
h1_roster

,name,number,position,overall
0,Triston Henry,9,g,0.72
1,Daniel Krutzen,10,d,0.71
2,Kwame Awuah,2,d,0.71
3,Dominic Samuel,22,d,0.65
4,Bertrand Owundi,16,d,0.48
5,Tristan Borges,4,m,0.71
6,Kyle Bekker,3,m,0.68
7,Giuliano Frano,7,m,0.49
8,Elimane Cissé,6,m,0.43
9,Christopher Nanco,13,f,0.43


In [76]:
def get_roster_overall(query,stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info): # use team stats to get the player information
    def get_score(data,name):
        db = data[data['name'] == name]
        db = db['overall'].values
        db = db[0]
        return db
    def get_image(data,name):
        db = data[data['name'] == name]
        if db['image'].empty:
            db = 'empty.jpg'
        else:
            db = db['image'].values
            db = db[0]
        return db
    roster = get_stats_all(stats,team_ref)
    roster = roster[roster['team'] == query].copy()
    roster = roster[['name','number','position']] # scale the dataframe down to what we need
    #roster.insert(3,'overall',a)
    a = []
    b = []
    for i in range(0,roster.shape[0]):
        #print('getting player ',roster.iloc[i]['name'])
        if roster.iloc[i]['position'] == 'f':
            a.append(get_score(rated_forwards,roster.iloc[i]['name']))
            b.append(get_image(player_info,roster.iloc[i]['name']))
        if roster.iloc[i]['position'] == 'm':
            a.append(get_score(rated_midfielders,roster.iloc[i]['name']))
            b.append(get_image(player_info,roster.iloc[i]['name']))
        if roster.iloc[i]['position'] == 'd':
            a.append(get_score(rated_defenders,roster.iloc[i]['name']))
            b.append(get_image(player_info,roster.iloc[i]['name']))
        if roster.iloc[i]['position'] == 'g':
            a.append(get_score(rated_keepers,roster.iloc[i]['name']))
            b.append(get_image(player_info,roster.iloc[i]['name']))
    roster['overall'] = a
    roster.insert(0,'image',b)
    #roster['image'] = b
    roster = index_reset(roster)
    return roster

In [77]:
roster = get_roster_overall('York9 FC',stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info)
roster

,image,name,number,position,overall
0,Austin-Ricci.jpg,Austin Ricci,7,f,0.09
1,Colm-Vance.jpg,Colm Vance,31,g,0.06
2,Cyrus-Rollocks.jpg,Cyrus Rollocks,17,f,0.06
3,Daniel-Gogarty.jpg,Daniel Gogarty,2,d,0.42
4,Diyaeddine-Abzi.jpg,Diyaeddine Abzi,20,d,0.72
5,Emilio-Estevez.jpg,Emilio Estevez,11,m,0.21
6,Emmanuel-Zambazis.jpg,Emmanuel Zambazis,14,m,0.17
7,Joseph-DiChiara.jpg,Joseph Di Chiara,8,m,0.47
8,Justin-Springer.jpg,Justin Springer,5,d,0.30
9,Kyle-Porter.jpg,Kyle Porter,19,m,0.55


In [78]:
schedule.iloc[3]['home']

'Valour FC'

In [79]:
game_info = schedule[schedule['home'] == q1]
game_info

,game,home,away
0,I1,Forge FC,York9 FC
9,I10,Forge FC,Cavalry FC
17,I18,Forge FC,FC Edmonton
22,I23,Forge FC,Valour FC
35,I36,Forge FC,FC Edmonton
41,I42,Forge FC,HFX Wanderers FC
44,I45,Forge FC,Valour FC
62,I63,Forge FC,Cavalry FC
6,I7,Forge FC,Pacific FC
69,I70,Forge FC,York9 FC


In [80]:
game_info.iloc[0]['game']

'I1'

In [81]:
game = game_info.iloc[0]['game']

In [82]:
game_info.iloc[0]['home']

'Forge FC'

In [83]:
q2 = game_info.iloc[0]['away']
q2

'York9 FC'

In [84]:
game_h = get_home_away_comparison(stats,game,q1)
game_a = get_home_away_comparison(stats,game,q2)

In [85]:
len(game_h)

14

In [86]:
len(game_a)

14

In [87]:
h1_roster = get_compare_roster(results,q1,team_stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers)
h1_roster

,name,number,position,overall
0,Triston Henry,9,g,0.72
1,Daniel Krutzen,10,d,0.71
2,Kwame Awuah,2,d,0.71
3,Dominic Samuel,22,d,0.65
4,Bertrand Owundi,16,d,0.48
5,Tristan Borges,4,m,0.71
6,Kyle Bekker,3,m,0.68
7,Giuliano Frano,7,m,0.49
8,Elimane Cissé,6,m,0.43
9,Christopher Nanco,13,f,0.43


In [88]:
h2_roster = get_compare_roster(results,q2,team_stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers)
h2_roster

,name,number,position,overall
0,Nathan Ingham,1,g,0.93
1,Luca Gasparotto,13,d,0.83
2,Diyaeddine Abzi,20,d,0.72
3,Morey Doner,3,d,0.66
4,Daniel Gogarty,2,d,0.42
5,Ryan Telfer,18,m,0.70
6,Manuel Aparicio,10,m,0.64
7,Kyle Porter,19,m,0.55
8,Joseph Di Chiara,8,m,0.47
9,Rodrigo Gattas,22,f,0.76


In [89]:
def get_team_history(data,query):
    df = data[data['away'] == query].copy()
    df = df[['d','m','as','hs','away','ar','home','hr']]
    df = df.rename(columns={'as':'hs','hs':'as','away':'home','ar':'hr','home':'away','hr':'ar'})
    db = data[data['home'] == query].copy()
    db = db[['d','m','hs','as','home','hr','away','ar']]
    db = pd.concat([db,df])
    db = db.tail(5)
    db = db.sort_values(by=['m','d'],ascending=False)
    return db

def get_five_game_form(data,query):
    db = get_team_history(data,query)
    db = db.pop('hr')
    a = []
    for i in db:
        if i == 'W':
            j = [1,0,0]
            a.append(j)
        if i == 'L':
            j = [0,1,0]
            a.append(j)
        if i == 'D':
            j = [0,0,1]
            a.append(j)
    db = pd.DataFrame(a,columns=['w','l','d'])
    db = pd.DataFrame(db.sum())
    return db

In [122]:
def power_rankings(results,team_ref):
    db = pd.DataFrame()
    a = []
    for name in team_ref['team']:
        form = get_five_game_form(results,name)
        a.append(form)
        #db['test'] = a
    #db = pd.DataFrame(a)
    return a

In [123]:
test = power_rankings(results,team_ref)
test

ValueError: Length of values does not match length of index

In [125]:
cfc = get_five_game_form(results,'Cavalry FC')
cfc

,0
w,2
l,2
d,1


In [90]:
team1_history = get_team_history(results,q1)
team1_history

,d,m,hs,as,home,hr,away,ar
99,2,11,1,0,Forge FC,W,Cavalry FC,L
91,12,10,0,4,Forge FC,L,York9 FC,W
90,9,10,1,2,Forge FC,L,Cavalry FC,W
84,2,10,1,0,Forge FC,W,FC Edmonton,L
76,18,9,1,1,Forge FC,D,HFX Wanderers FC,D


In [91]:
team1_history = get_five_game_form(results,q1)

In [92]:
team1_history

,0
w,2
l,2
d,1


In [97]:
for i in team1_history:
    if i == 'W':
        print(3)
    if i == 'L':
        print(0)
    if i == 'D':
        print(1)

In [98]:
team2_history = get_team_history(results,q2)
team2_history

,d,m,hs,as,home,hr,away,ar
93,16,10,4,0,York9 FC,W,Valour FC,L
88,6,10,0,1,York9 FC,L,Forge FC,W
78,21,9,3,1,York9 FC,W,FC Edmonton,L
71,11,9,2,0,York9 FC,W,Pacific FC,L
69,8,9,1,2,York9 FC,L,Forge FC,W


In [99]:
test = get_long_name('FFC',team_ref)

In [100]:
test

'Forge FC'

In [105]:
get_games = results['game']
if year == '2019':
    get_games = get_games[:-2]

In [106]:
def get_game_roster_prediction(stats,get_games,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,results,team_stats,team_ref):
    a = []
    for game in get_games: # cycle through the available games
        row = results[results['game'] == game] # select specific game results
        for team in row.iloc[0][['home','away']]: # cycle through the teams for each result
            if row.iloc[0]['home'] == team:
                result = row.iloc[0]['hr'] # get the appropriate result for each team
            else:
                result = row.iloc[0]['ar'] 
            if result == 'W': # alter the value for the model classifier
                result = 3
            elif result == 'D':
                result = 2
            else:
                result = 1
            game_check = get_home_away_comparison(stats,game,team)# get the roster for the team in the game
            # get the player overall score for each player in the game
            game_roster = get_compare_roster(team,game_check,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,results,team_stats,team_ref)
            game_roster = index_reset(game_roster)
            b = []
            b.append(game) # collecting all the information in the list
            b.append(team)
            for i in range(game_roster.shape[0]):
                b.append(game_roster.iloc[i]['overall']) # get the player overall score for each player in the game
            if len(b) < 16:
                i = int(16 - len(b))
                for j in range(0,i):
                    b.append(0)
            b.append(int(result))
            a.append(b)
    return a

In [178]:
array = get_game_roster_prediction(stats,get_games,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,results,team_stats,team_ref)

In [187]:
db = pd.DataFrame(array,columns=['game','team','p1','p2','p3','p4','p5','p6','p7','p8','p9','p9','p10','p11','p12','p13','r'])

In [188]:
db.shape

(196, 17)

In [181]:
def pump_it_up(db):
    df = db.copy()
    dc = df.copy()
    m = df['p1'].copy()
    n = df['p2'].copy()
    o = df['p3'].copy()
    p = df['p4'].copy()
    q = df['p5'].copy()
    r = df['p6'].copy()
    df['p1'] = dc.pop('p8')
    df['p2'] = dc.pop('p10')
    df['p3'] = dc.pop('p12')
    df['p4'] = dc.pop('p9')
    df['p5'] = dc.pop('p11')
    df['p6'] = dc.pop('p13')
    df['p7'] = m
    df['p8'] = n
    df['p9'] = o
    df['p10'] = p
    df['p11'] = q
    df['p12'] = r
    df['p13'] = dc.pop('p7')
    dc = df.copy()
    db = pd.concat([db,df])
    df = dc.copy()
    m = df['p13'].copy()
    n = df['p12'].copy()
    o = df['p11'].copy()
    p = df['p10'].copy()
    q = df['p9'].copy()
    r = df['p8'].copy()
    df['p13'] = dc.pop('p8')
    df['p12'] = dc.pop('p10')
    df['p11'] = dc.pop('p12')
    df['p10'] = dc.pop('p9')
    df['p9'] = dc.pop('p11')
    df['p8'] = dc.pop('p13')
    df['p7'] = m
    df['p6'] = n
    df['p5'] = o
    df['p4'] = p
    df['p3'] = q
    df['p2'] = r
    df['p1'] = dc.pop('p7')
    #dc = df.copy()
    db = pd.concat([db,df])
    db = index_reset(db)
    return db

In [106]:
def pump_it_up(db):
    cols = ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11','p12','p13']
    np.random.shuffle(cols)
    df = db.copy()
    dc = df.copy()
    m = df[cols[0]].copy()
    n = df[cols[1]].copy()
    o = df[cols[2]].copy()
    p = df[cols[3]].copy()
    q = df[cols[4]].copy()
    r = df[cols[5]].copy()
    df[cols[0]] = dc.pop(cols[7])
    df[cols[1]] = dc.pop(cols[8])
    df[cols[2]] = dc.pop(cols[9])
    df[cols[3]] = dc.pop(cols[10])
    df[cols[4]] = dc.pop(cols[11])
    df[cols[5]] = dc.pop(cols[12])
    df[cols[6]] = m
    df[cols[7]] = n
    df[cols[8]] = o
    df[cols[9]] = p
    df[cols[10]] = q
    df[cols[11]] = r
    df[cols[12]] = dc.pop(cols[6])
    dc = df.copy()
    db = pd.concat([db,df])
    '''df = dc.copy()
    np.random.shuffle(cols)
    m = df[cols[0]].copy()
    n = df[cols[1]].copy()
    o = df[cols[2]].copy()
    p = df[cols[3]].copy()
    q = df[cols[4]].copy()
    r = df[cols[5]].copy()
    df[cols[0]] = dc.pop(cols[7])
    df[cols[1]] = dc.pop(cols[8])
    df[cols[2]] = dc.pop(cols[9])
    df[cols[3]] = dc.pop(cols[10])
    df[cols[4]] = dc.pop(cols[11])
    df[cols[5]] = dc.pop(cols[12])
    df[cols[6]] = m
    df[cols[7]] = n
    df[cols[8]] = o
    df[cols[9]] = p
    df[cols[10]] = q
    df[cols[11]] = r
    df[cols[12]] = dc.pop(cols[6])
    #dc = df.copy()
    db = pd.concat([db,df])'''
    db = index_reset(db)
    return db

In [189]:
df = pump_it_up(db)
df.shape

(588, 17)

In [190]:
db = pump_it_up(df)
db.shape

(1764, 17)

In [191]:
df = pump_it_up(db)
df.shape

(5292, 17)

In [192]:
db = pump_it_up(df)
db.shape

(15876, 17)

In [113]:
db.pop('game')
db.pop('team')
print('done')

done


In [114]:
y = db.pop('r')
X = db

In [115]:
#importing libraries from sklearn
from sklearn import tree
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler#,Imputer
from sklearn import metrics
#colours = sns.set_palette('pastel')

In [116]:
# import algorithm modules
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [118]:
seed = 7

In [119]:
#DecisionTreeRegressor
def decisionTree(x,y):
    model = DecisionTreeClassifier(criterion='gini',
                                   splitter='random',
                                   max_depth=8,
                                   max_features='log2')
    model.fit(x, y)
    return model

dt = decisionTree(X_train, y_train)

#K Neighbors Classifier
# algorithm = ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’
def kNeighbors(x,y):
    model = KNeighborsClassifier(algorithm = 'auto',
                                 leaf_size=30,
                                 metric='minkowski',
                                 metric_params=None,
                                 n_jobs=1,
                                 n_neighbors=5,
                                 p=2,
                                 weights='uniform')
    model.fit(x, y)
    return model
    
knn = kNeighbors(X_train, y_train)

#Create a Gaussian Classifier
def gaussianNB(x,y):
    model = GaussianNB(priors=None,
                       var_smoothing=1e-09)
    model.fit(x, y)
    return model
    
gnbC = gaussianNB(X_train, y_train)

In [121]:
#Random Forest Regression
def forestRegression_1(x,y):
    model = RandomForestClassifier(n_estimators = 100,
                                   min_samples_leaf = 5,
                                   min_samples_split = 12,
                                   random_state = 0,
                                   max_depth = 80)
    model.fit(x, y)
    return model
    
rf1 = forestRegression_1(X_train, y_train)

#Create a svm Classifier
def SVM_1(x,y):
    model = svm.SVC(kernel='poly',
                    degree=8,
                    gamma ='auto',
                    probability=True)
    # kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
    # gamma = ‘scale’, ‘auto’
    model.fit(x, y)
    return model

svm1 = SVM_1(X_train, y_train)

In [122]:
# Bagged Decision Trees for Classification - necessary dependencies
def baggedTree(x,y,seed):
    model = BaggingClassifier(base_estimator=rf1,
                              n_estimators=10,
                              random_state=seed)
    model.fit(x, y)
    return model

bag = baggedTree(X_train, y_train,seed)

# AdaBoost Classification
def adaBoost(x,y,seed):
    model = AdaBoostClassifier(n_estimators=70,
                               random_state=seed,
                               algorithm='SAMME')
    model.fit(x, y)
    return model
    
ada = adaBoost(X_train, y_train,seed)

# Voting Ensemble for Classification
def ensembleClassifier(x,y):
    # create the sub models
    estimators = []
    estimators.append(('rf1', rf1))
    estimators.append(('knn', knn))
    estimators.append(('bag', bag))

    # create the ensemble model
    model = VotingClassifier(estimators,voting='soft',weights=[0.83,0.79,0.82])
    model.fit(x,y)
    return model

ens = ensembleClassifier(X_train, y_train)

In [123]:
def check(a,b):
    if a == b:
        result = '<'
    else:
        result = '-'
    return result

In [124]:
print('rf1',rf1.score(X_train, y_train))
print('ens',ens.score(X_train, y_train))
print('bag',bag.score(X_train, y_train))
print('knn',knn.score(X_train, y_train))

rf1 0.7283599088838268
ens 0.6789293849658314
bag 0.6652619589977221
knn 0.5865603644646925


In [125]:
random_nums = np.random.randint(low=1, high=58, size=(20))

In [126]:
def print_pred_results(model,result,num):
    print('model          : ', result, check(result,y_test.iloc[num]))

def predictionTest(num,model1,model2,model3):
    p = X_test.iloc[num].tolist()
    result1 = model1.predict([p]).flatten()
    result2 = model2.predict([p]).flatten()
    result3 = model3.predict([p]).flatten()
    print('\nActual       : ',y_test.iloc[num])
    print_pred_results(model1,result1,num)
    print_pred_results(model2,result2,num)
    print_pred_results(model3,result3,num)

In [127]:
for i in random_nums:
    predictionTest(i,rf1,knn,bag)


Actual       :  2
model          :  [1] -
model          :  [1] -
model          :  [1] -

Actual       :  1
model          :  [1] <
model          :  [1] <
model          :  [1] <

Actual       :  3
model          :  [3] <
model          :  [3] <
model          :  [3] <

Actual       :  1
model          :  [1] <
model          :  [1] <
model          :  [1] <

Actual       :  1
model          :  [1] <
model          :  [1] <
model          :  [1] <

Actual       :  3
model          :  [1] -
model          :  [1] -
model          :  [1] -

Actual       :  3
model          :  [3] <
model          :  [2] -
model          :  [3] <

Actual       :  1
model          :  [1] <
model          :  [2] -
model          :  [1] <

Actual       :  3
model          :  [3] <
model          :  [3] <
model          :  [3] <

Actual       :  1
model          :  [1] <
model          :  [3] -
model          :  [3] -

Actual       :  3
model          :  [1] -
model          :  [1] -
model          :  [1] -

In [153]:
def cycle_prob_test(num,model):
    p = X_test.iloc[num].tolist()
    e = model.predict_proba([p]).flatten()
    return e.tolist()

def cycle_pred_test(num,model):
    p = X_test.iloc[num].tolist()
    e = model.predict([p]).flatten()
    if e == y_test.iloc[num]:
        a = 1
    else:
        a = 0
    return a

def model_pred_test(model):
    b = []
    prob = []
    for i in random_nums:
        b.append(cycle_pred_test(i,model))
        prob.append(cycle_prob_test(i,model))
    dz = pd.DataFrame(b)
    df = pd.DataFrame(prob)
    c = str(float(dz.sum().values / 20))
    print('score :',c)
    #print(df)
    #return df

In [154]:
test_check = model_pred_test(knn)

score : 0.5


In [155]:
model_pred_test(ada)

score : 0.65


In [156]:
model_pred_test(bag)

score : 0.65


In [157]:
model_pred_test(rf1)

score : 0.7


In [158]:
model_pred_test(ens)

score : 0.7


In [135]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import matplotlib.pyplot as plt

print(tf.__version__)

2.1.0


In [136]:
# import TENSORFLOW and KERAS libraries

import tensorflow as tf

from tensorflow import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, LeakyReLU, BatchNormalization, Dropout
from keras.layers.merge import concatenate
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD

Using TensorFlow backend.


In [137]:
# Insert Hyperparameters
LEARNRATE = 0.001
EPOCHS = 20
BATCH_SIZE = 128

In [138]:
INPUT_SHAPE = X_train.shape[1] # number of features - get the len() number of the dataset's columns
DENSE_64 = INPUT_SHAPE * 64
DENSE_32 = INPUT_SHAPE * 32
DENSE_16 = INPUT_SHAPE * 16
DENSE_8 = INPUT_SHAPE * 8
DENSE_4 = INPUT_SHAPE * 4
DENSE_2 = INPUT_SHAPE * 2
DENSE_1 = INPUT_SHAPE
DENSE_CUT = int(INPUT_SHAPE / 2)

print('input:',INPUT_SHAPE,
      '\ndense layers',
      '\n64   :',DENSE_64,
      '\n32   :',DENSE_32,
      '\n16   :',DENSE_16,
      '\n8    :',DENSE_8,
      '\n4    :',DENSE_4,
      '\n2    :',DENSE_2,
      '\n1    :',DENSE_1,
      '\ncut  :',DENSE_CUT)

input: 14 
dense layers 
64   : 896 
32   : 448 
16   : 224 
8    : 112 
4    : 56 
2    : 28 
1    : 14 
cut  : 7


In [139]:
def nNet():
    vis = Input(shape=(INPUT_SHAPE,))
    h1 = Dense(DENSE_64, activation='relu', name = "Hidden_Layer_1")(vis)
    h2 = Dense(DENSE_64, activation='relu', name = "Hidden_Layer_2")(h1)
    h3 = Dense(DENSE_32, activation='relu', name = "Hidden_Layer_3")(h2)
    h4 = Dense(DENSE_16, activation='relu', name = "Hidden_Layer_4")(h3)
    out = Dense(DENSE_1, activation='softmax', name = "Output_Layer")(h4)
    model = Model(vis, out)
    return model

In [140]:
tfn = nNet()
tfn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14)                0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 896)               13440     
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 896)               803712    
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 448)               401856    
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 224)               100576    
_________________________________________________________________
Output_Layer (Dense)         (None, 14)                3150      
Total params: 1,322,734
Trainable params: 1,322,734
Non-trainable params: 0
_________________________________________________

In [141]:
tfn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['mae','mse','accuracy'])

In [142]:
tfn_history = tfn.fit(X_train,
                  y_train,
                  epochs = EPOCHS,
                  verbose = 1)

Epoch 1/20
8780/8780 [==============================] - 12s 1ms/step - loss: 2.3743 - mae: 1.9251 - mse: 4.5367 - accuracy: 0.0000e+00
Epoch 2/20
8780/8780 [==============================] - 11s 1ms/step - loss: 2.3703 - mae: 1.9251 - mse: 4.5381 - accuracy: 0.0000e+00
Epoch 3/20
8780/8780 [==============================] - 11s 1ms/step - loss: 2.3703 - mae: 1.9251 - mse: 4.5381 - accuracy: 0.0000e+00
Epoch 4/20
8780/8780 [==============================] - 11s 1ms/step - loss: 2.3703 - mae: 1.9251 - mse: 4.5381 - accuracy: 0.0000e+00
Epoch 5/20
8780/8780 [==============================] - 11s 1ms/step - loss: 2.3703 - mae: 1.9251 - mse: 4.5381 - accuracy: 0.0000e+00
Epoch 6/20
8780/8780 [==============================] - 11s 1ms/step - loss: 2.3703 - mae: 1.9251 - mse: 4.5381 - accuracy: 0.0000e+00
Epoch 7/20
8780/8780 [==============================] - 11s 1ms/step - loss: 2.3703 - mae: 1.9251 - mse: 4.5381 - accuracy: 0.0000e+00
Epoch 8/20
8780/8780 [==============================] -

In [143]:
test_loss, test_acc = tfn.evaluate(X_train,
                                   y_train,
                                   verbose=2)
print('\nTest accuracy:', test_acc)

ValueError: too many values to unpack (expected 2)